In [94]:
import os
import csv
import boto3
import pandas as pd
import time
from tqdm import tqdm

confidence = 90
path = "upload/"

In [95]:
def convert(photo):
    with open(path + photo,'rb') as source_image:
        source_bytes = source_image.read()
    return source_bytes

In [96]:
with open('credentials.csv','r') as input:
    next(input)
    reader = csv.reader(input)
    for line in reader:
        access_key_id = line[2]
        secret_access_key = line[3]

In [97]:
t0 = time.time()
client = boto3.client('rekognition',
                      region_name='us-east-1',
                      aws_access_key_id = access_key_id,
                      aws_secret_access_key = secret_access_key)

In [98]:
photos = [f for f in os.listdir(path) if (f.lower().endswith(".jpg") or f.lower().endswith(".png"))]
df_dir = pd.DataFrame(photos, columns=['file'])
df_dir['bytes'] = df_dir['file'].apply(convert)
df_dir.set_index('file', inplace=True)

In [91]:
#bucket
#s3 = boto3.resource('s3',
#                    region_name='us-east-1',
#                    aws_access_key_id = access_key_id,
#                    aws_secret_access_key = secret_access_key)

#bucket = s3.Bucket('project-identify')

#photos = []
#for file in bucket.objects.all():
#    photos.append(file.key)

In [92]:
file = []
#photos = ['0009-IMG_0027.JPG'] #bucket
for photo in tqdm(photos):
    
    source_bytes = df_dir.loc[photo].bytes

    response = client.detect_text(Image={'Bytes':source_bytes}, Filters={'WordFilter': {'MinConfidence': confidence} })
    #response = client.detect_text(Image={'S3Object': { 'Bucket': 'project-identify', 'Name': photo} }, 
    #                              Filters={'WordFilter': {'MinConfidence': confidence} })

    textDetections = response['TextDetections']
    bibs = []
    bibs.append(photo)
    for text in textDetections:
        if text['DetectedText'].isnumeric() and text['Type'] == 'LINE':
            bibs.append(text['DetectedText'])
            #print(text['DetectedText'] + ", " + str(text['Confidence']))

    file.append(bibs)

  3%|██▊                                                                              | 26/745 [00:52<23:58,  2.00s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(file)

In [ ]:
df.to_csv('file.csv', index=False, header=False, sep=';')

In [ ]:
t1 = time.time()

In [ ]:
total = t1-t0
print(total/60)

In [ ]:
col = ['file']
for i in range(df.columns.stop-1):
    col.append('bib'+str(i+1))

In [ ]:
df.columns = col

In [ ]:
df.head()

In [ ]:
round((df.bib1.count() / df.file.count()) * 100)